###

### Import Required Library

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
#load the survey dataset
df = pd.read_csv('SCFP2022.csv')


In [29]:
df.shape

(22975, 356)

* This dataset has over 300 variables, we only need just a few of this for our clustering excerise.
* Let's filter our dataset to get only business owners and those with income less than $500_000

In [30]:
df.head(3)

,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
0,1,11,3027.956120,2,70,5,9,3,2,2,...,4,2,4,2,1,8,3,3,2,1
1,1,12,3054.900065,2,70,5,9,3,2,2,...,4,2,5,2,1,8,3,3,2,1
2,1,13,3163.637766,2,70,5,9,3,2,2,...,4,2,4,2,1,8,3,3,1,1


In [5]:
# Check the proportion of business owner
buss_owner= df.HBUS.value_counts(normalize=True)
buss_owner

HBUS
0    0.722046
1    0.277954
Name: proportion, dtype: float64

* They are approximately 28% of the entire survey.

In [6]:
df['INCOME']

0        38804.734469
1        38264.278557
2        36102.454910
3        33508.266533
4        35561.998998
             ...     
22970    38912.825651
22971    33508.266533
22972    38912.825651
22973    38912.825651
22974    35670.090180
Name: INCOME, Length: 22975, dtype: float64

In [7]:
#create mask for business owner and those with income less than 500000

mask = (df["HBUS"]==1) & (df['INCOME'] < 500_000)
df= df[mask]
df.shape

(3320, 356)

In [8]:
# we are going to select the desired 10 variables using their variance
df1=df.var().sort_values().tail(5)
df1

BUS         2.429200e+15
NHNFIN      3.520996e+15
NFIN        3.554988e+15
NETWORTH    4.609543e+15
ASSET       4.702561e+15
dtype: float64

In [9]:
# fig = px.bar(x=df1/1e6, y=df1.index)
# fig.show()

In [10]:
# trim_df = df.apply(trimmed_var).sort_values().tail(10)
# trim_df

In [11]:
#column to use for our cluster
col= df.var().sort_values().tail(5).index.to_list()
col

['BUS', 'NHNFIN', 'NFIN', 'NETWORTH', 'ASSET']

In [12]:
X=df[col]

In [13]:
X.head(3)

,BUS,NHNFIN,NFIN,NETWORTH,ASSET
5,460000,520000.0,1100000.0,721800.0,1240500.0
6,460000,520000.0,1100000.0,723800.0,1242500.0
7,460000,520000.0,1100000.0,721400.0,1241100.0


In [14]:
n_clusters = range(2, 10)
inertia= []
silhouette = []

for k in  n_clusters:

    model= make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    model.fit(X)
    inertia.append(model.named_steps['kmeans'].inertia_)
    silhouette.append(silhouette_score(X, model.named_steps['kmeans'].labels_) )

print("inertia_errors len:", len(inertia))
print("Inertia:", inertia)
print()
print("silhouette_scores len:", len(silhouette))
print("Silhouette Scores:", silhouette)

c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\cluster\_kmeans.py:1416:

inertia_errors len: 8
Inertia: [2258.6061813449705, 1034.6948915845476, 603.3356274917558, 367.7459742639467, 267.44400821975773, 175.11461027372417, 127.86276174363974, 99.86989137654577]

silhouette_scores len: 8
Silhouette Scores: [0.9877503488640621, 0.9866918241180391, 0.9386359959240038, 0.9383389019655799, 0.9388938268490268, 0.862342946741767, 0.8485608818302282, 0.7882820602231311]


In [15]:
fig = px.line(x=n_clusters, y=inertia, title="Inertia VS Number of Clusters")

fig.show()

In [16]:
fig = px.line(x=n_clusters, y=silhouette , title="Silhouette VS Number of Clusters")

fig.show()

#### The Optimal number of cluster from the inertia and silouette plot is 4

### Build The Final Model

In [17]:
finalModel =make_pipeline(StandardScaler(), KMeans(n_clusters=4, random_state=42))
finalModel.fit(X)
labels= finalModel.named_steps['kmeans'].labels_

c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [33]:
import pickle
with open("modelfile.pkl", "wb") as f:
    model =pickle.dump(finalModel, f)
    

In [35]:
#make prediction
with open("modelfile.pkl", "rb") as f:
    model = pickle.load(f)

In [36]:
type(model)

sklearn.pipeline.Pipeline

In [41]:
test= np.array([460000, 520000, 110000, 721800, 1240500])
model.predict(test.reshape(1, -1))[0]

c:\Users\Techa\Desktop\2023leave\mlflow\mlflowenv\lib\site-packages\sklearn\base.py:465: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



2

In [19]:
xgb = X.groupby(labels).mean()

print("xgb type:", type(xgb))
print("xgb shape:", xgb.shape)
xgb

xgb type: <class 'pandas.core.frame.DataFrame'>
xgb shape: (4, 5)


,BUS,NHNFIN,NFIN,NETWORTH,ASSET
0,6.407702e+07,6.922363e+07,7.201959e+07,9.724490e+07,9.866078e+07
1,5.360750e+08,5.426301e+08,5.453758e+08,6.442533e+08,6.443268e+08
2,1.266298e+06,1.715672e+06,2.408951e+06,3.298621e+06,3.577149e+06
3,7.024600e+08,1.092665e+09,1.097909e+09,1.180558e+09,1.206008e+09


In [20]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(xgb, barmode='group')

fig.show()

In [21]:
#Create two Principal Components

In [22]:
pca = PCA(n_components=2, random_state=42)
xpca = pca.fit_transform(X)
pcadf= pd.DataFrame(xpca, columns=["PC1","PC2"])
pcadf.head(2)

,PC1,PC2
0,-1.732188e+07,755212.879415
1,-1.731988e+07,753886.480472


In [23]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(data_frame=pcadf, x='PC1', y='PC2', color=labels.astype(str))

fig.show()